<a href="https://colab.research.google.com/github/12NaN/data-science-project/blob/master/project_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline


In [0]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)

In [3]:
train_df.sample(5).transpose()

rental_id,7267399,7187728,7313305,7409185,7338100
addr_unit,#B,#10E,#5F,#32A,#1D
building_id,60169,355315,548278,54790,78889
bedrooms,1,3,2,1,1
bathrooms,1,2,1,1,1
size_sqft,900,1250,1119,620,0
created_at,2018-06-12 22:16:35,2018-05-17 16:59:05,2018-06-27 17:00:31,2018-07-25 18:02:53,2018-07-06 11:28:53
addr_street,323 WEST 75 STREET,185 SOUTH 4 STREET,40 LINDEN BOULEVARD,560 WEST 43 STREET,315 EAST 74 STREET
addr_city,New York,Brooklyn,Brooklyn,New York,New York
addr_zip,10023,11211,11226,10036,10021
addr_lat,40.7824,40.7112,40.6519,40.7609,40.77


In [0]:
submit1_df['min_to_subway'] = submit1_df['min_to_subway'].fillna(submit1_df['min_to_subway'].mean())
submit1_df.year_built = submit1_df.year_built.fillna(submit1_df.year_built.median(), axis=0)


In [0]:
# extract usable features
feature_cols = [
    'bedrooms', 'year_built', 'bathrooms', 'min_to_subway', 
    'size_sqft', 'no_fee', 'has_doorman'
]
train_features = train_df[feature_cols] 

# impute missing values with medians
train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
train_target = train_df['rent']

In [5]:
lreg = LinearRegression()
lreg.fit(train_features, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:
test_features = test_df[feature_cols] 

# impute missing values with medians
test_features = test_features.fillna(train_features.median(), axis=0)

# construct predictions 
test_df['predicted'] = lreg.predict(test_features)

mean_squared_error(test_df['rent'], test_df['predicted'])

3775738.256385183

In [7]:
master_df = train_df.append(test_df, sort=False)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']

lreg.fit(master_features, master_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
submit1_features = submit1_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit1_df['predictions'] = lreg.predict(submit1_features)
submit1_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit1_df['fake_rent'] = np.ones(submit1_df['predictions'].shape) * master_target.median()
mean_squared_error(submit1_df['predictions'], submit1_df['fake_rent'])


4608738.849344819

In [0]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_features, train_target);

In [77]:

test_features = test_df[feature_cols] 
test_features = test_features.fillna(train_features.median(), axis=0)
submit1_df['predictions'] = rf.predict(submit1_features)
# Probabilities for each cla
print('Mean squared Error:', mean_squared_error(test_df['rent'], submit1_df['predictions']))

Mean squared Error: 13647530.009061173


In [58]:
submit1_df.isna().sum()


addr_unit                   29
building_id                  0
bedrooms                     0
bathrooms                    0
size_sqft                    0
created_at                   0
addr_street                  0
addr_city                    0
addr_zip                     0
addr_lat                     0
addr_lon                     0
bin                          0
bbl                          0
floor_count                  0
year_built                   0
min_to_subway                0
has_doorman                  0
has_elevator                 0
has_fireplace                0
has_dishwasher               0
is_furnished                 0
has_gym                      0
allows_pets                  0
has_washer_dryer             0
has_garage                   0
has_roofdeck                 0
has_concierge                0
has_pool                     0
has_garden                   0
has_childrens_playroom       0
rent                      2000
no_fee                       0
descript